In [ ]:
!pip install torch lightning numpy kaggle wandb torch-geometric polars


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# Carica il file kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexxxyy47","key":"2c3d331934a8a1d64227f77606e63d4c"}'}

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d ealaxi/paysim1
!unzip paysim1.zip
!rm paysim1.zip

 98% 175M/178M [00:01<00:00, 165MB/s]
100% 178M/178M [00:01<00:00, 138MB/s]
Archive:  paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [ ]:
import pandas as pd, sys, plotly.graph_objects as go, plotly.express as px, numpy as np, torch, random as rnd, torch.nn as nn, lightning as L, wandb as wndb, pdb
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from matplotlib import pyplot as plt
import networkx as nx
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch import optim
from torchmetrics.classification import BinaryF1Score
from torch_geometric.nn import MessagePassing
from typing import Optional
from torch.nn import Parameter
from torch_geometric.typing import (
    Adj,
    OptPairTensor,
    OptTensor,
    SparseTensor,
    torch_sparse,
)
from torch_geometric.utils import add_remaining_self_loops
from torch_geometric.utils import add_self_loops as add_self_loops_fn
from torch_geometric.utils import (
    is_torch_sparse_tensor,
    scatter,
    spmm,
    to_edge_index,
)
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.utils.sparse import set_sparse_value
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import zeros
import polars as pl, json




In [ ]:
# PARAMETERS

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ACCELERATOR =  "gpu" if torch.cuda.is_available() else "cpu"
SEED = 42

rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEPTH = 2
LR = 1e-3
WD = 1e-5
H_SIZE = 8

"aaf831dabc88d936d4e6b439b798bb4cb42814ea"

#wndb.login()



'aaf831dabc88d936d4e6b439b798bb4cb42814ea'

In [ ]:
# UTILS FUNCTIONS

def load_dataframe( dataset_file : str):
    return pl.read_csv(dataset_file)


def find_null_or_empty_records( dataframe):
    return dataframe.filter(df.apply(lambda col: col.is_nan().any()))

def print_progress_bar(percentuale, lunghezza_barra=20):
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% completo")
    sys.stdout.flush()


def compute_kind_inconsistence(dataframe):
    return {"inconsistent orig balance": len(dataframe.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount')))/len(dataframe),
            "inconsistent dest balance": len(dataframe.filter(abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount')))/len(dataframe),
            "zero cash transaction": len(dataframe.filter(pl.col('amount') == 0 ))/len(dataframe),
            "self-transaction": len(dataframe.filter(pl.col('nameOrig') == pl.col('nameDest')))/len(dataframe)
            }

def plot_histogram(to_plot):


    # Converti il dizionario in un array di valori
    values = list(to_plot.values())

    # Crea un istogramma
    fig = go.Figure(data=[go.Bar(x=list(to_plot.keys()), y=values)])

    # Mostra l'istogramma
    fig.show()



def plot_categories(dataframe):
    # Calcola la frequenza di ogni categoria nella colonna 'type'
    counts = dataframe.groupby('type').count()

    # Crea l'istogramma con Plotly Express
    fig = px.bar(counts, x='type', y='count', title='Istogramma delle categorie nella colonna "type"')

    # Mostra il plot
    fig.show()


def build_names_conversion_index(df):
  all_names = df.with_column(pl.col("nameOrig").concat(pl.col("nameDest")), 'unique_names').select('unique_names').distinct()

  # Costruisci il dizionario di mapping
  name_to_id = {nome: id for id, nome in enumerate(df['unique_names'].to_list())}


  #breakpoint()
  return name_to_id


def compute_mean_var(df, name,var=False):
  orig = df.filter(pl.col('nameOrig') == name).select(pl.col('newbalanceOrig'))['newbalanceOrig']
  dest = df.filter(pl.col('nameDest') == name).select(pl.col('newbalanceDest'))['newbalanceDest']
  if var:
    return (len(orig) * orig.var() + len(dest) * dest.var()) / (len(orig) + len(dest))

  else:
    return (orig.mean() + dest.mean()) / 2






In [ ]:
class FraudDetectionDataset(Dataset):

    def __init__(self,dataset_file : str):
        dataframe = load_dataframe(dataset_file)
        dataframe = dataframe.select(pl.exclude("isFlaggedFraud"))
        dataframe = dataframe.with_column(pl.col("isFraud").cast(int).alias("isFraud"))
        self.raw_data = dataframe
        self.transaction_types = {
            "CASH_IN": 0,
            "CASH_OUT": 1,
            "DEBIT": 2,
            "PAYMENT": 3,
            "TRANSFER": 4
        }

    def analize_data(self,find_empty_records = True):
        print("----HEAD----")
        print(self.raw_data.head())
        print("----INFO----")
        print(self.raw_data.info())
        print("----DESCRIBE----")
        print(self.raw_data.describe())
        if find_empty_records:
          find_null_or_empty_records(self.raw_data)

    def extract_inconsistent_transactions(self):

        return self.raw_data.filter(
            abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount') |
            abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount') | pl.col('amount') == 0 | pl.col('nameOrig') == pl.col('nameDest')
        )


    def remove_inconsistent(self):
        self.raw_data = self.raw_data.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) == abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) & pl.col('amount') != 0)


    def convert_type(self):
      self.raw_data = self.raw_data.with_column(pl.col("type").map(lambda x: self.transaction_types[x]).alias("type"))



    def build_graph(self, val_prc, test_prc):
      edge_index = []
      #breakpoint()
      node_names = build_names_conversion_index(self.raw_data)
      id = 0
      x = []
      self.raw_data =  self.raw_data.with_column(pl.col("nameOrig").map(lambda x: node_names[x]).alias("nameOrig"))
      self.raw_data =  self.raw_data.with_column(pl.col("nameDest").map(lambda x: node_names[x]).alias("nameDest"))
      self.raw_data =  self.raw_data.with_column(pl.col("step").map(lambda x: x%24).alias("step"))

      nodes = list(node_names.items())

      x_dataframe = pl.DataFrame({"node_name": nodes[0],
                                  "node_id": nodes[1]})
      x_dataframe.with_column(
                              pl.col("node_name").map(lambda x: 1 if x[0].startswith('M') else 0 ).alias("commercial"),
                              pl.col("node_name").map(lambda x: compute_mean_var(self.raw_data,x) ).alias("mean"),
                              pl.col("node_name").map(lambda x: compute_mean_var(self.raw_data,x, var=True) ).alias("var")
                              )
      edge_attr = self.raw_data.with_column(pl.col('step').zip_with(pl.col('type'), pl.col('amount'), lambda x, y, z: int(str(x) + str(z) + str(y)) ).alias('edge_attr')).select('edge_attr')
      edge_attr = torch.tensor(edge_attr['edge_attr'], dtype=torch.float16)
      y = torch.tensor(self.raw_data['isFraud'], dtype=torch.float16)
      combined_df = self.raw_data.select(['nameOrig', 'nameDest']).hstack()

      edge_index = torch.tensor(combined_df.to_numpy(), dtype=torch.int).t()
      combined_df = self.raw_data.select(['commercial', 'mean', 'var']).hstack()
      x = torch.tensor(combined_df.to_numpy(), dtype=torch.float16)
      self.raw_data = None

      json_data = {
          "edge_attr": edge_attr.tolist()
          "y": y.tolist()
          "edge_index": edge_index.tolist()
          "x": x.tolist()
      }

      with open('graph_data.json', 'w') as f:
        json.dump(json_data,f)



      #breakpoint()
      data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
      data.y = y
      self.split = T.RandomNodeSplit(num_val=val_prc, num_test=test_prc)
      self.data = [self.split(data)]


    def __getitem__(self, index):
      return self.data[index]

    def __len__(self):
      return 1


    def collate(self, data):

      #breakpoint()
      return data[0]



    def get_dataloader(self):
      return DataLoader(self, batch_size=1, shuffle=False, collate_fn = self.collate)


    def to_device(self):
      #breakpoint()
      self.data[0].x.to(DEVICE)
      self.data[0].y.to(DEVICE)
      self.data[0].edge_attr.to(DEVICE)
      self.data[0].edge_index.to(DEVICE)
      self.data[0].train_mask.to(DEVICE)
      self.data[0].val_mask.to(DEVICE)
      self.data[0].test_mask.to(DEVICE)









In [ ]:
dataset = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv")
#inconsistent_data = dataset.extract_inconsistent_transactions()

In [ ]:
d = compute_kind_inconsistence(inconsistent_data)
plot_histogram(d)
names_distinct = set(dataset.raw_data["nameOrig"]).union(set(dataset.raw_data["nameDest"]))

In [ ]:
print(dataset.raw_data.columns)
dataset.convert_type()
dataset.build_graph(0.1,0.2)




Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')


In [ ]:
def convert_to_networkx(graph):
	g = to_networkx(graph, node_attrs=['x'])

	return g

def plot_graph(g):
	plt.figure(figsize=(9,7))
	nx.draw_spring(g, node_size=30, arrows=False)
	plt.show()



g = convert_to_networkx(dataset.data[0])
plot_graph(g)
del g


In [ ]:
@torch.jit._overload
def gcn_norm(  # noqa: F811
        edge_index, edge_weight, num_nodes, improved, add_self_loops, flow,
        dtype):
    # type: (Tensor, OptTensor, Optional[int], bool, bool, str, Optional[int]) -> OptPairTensor  # noqa
    pass


@torch.jit._overload
def gcn_norm(  # noqa: F811
        edge_index, edge_weight, num_nodes, improved, add_self_loops, flow,
        dtype):
    # type: (SparseTensor, OptTensor, Optional[int], bool, bool, str, Optional[int]) -> SparseTensor  # noqa
    pass


def gcn_norm(  # noqa: F811
    edge_index: Adj,
    edge_weight: OptTensor = None,
    num_nodes: Optional[int] = None,
    improved: bool = False,
    add_self_loops: bool = True,
    flow: str = "source_to_target",
    dtype: Optional[torch.dtype] = None,
):
    fill_value = 2. if improved else 1.

    if isinstance(edge_index, SparseTensor):
        assert edge_index.size(0) == edge_index.size(1)

        adj_t = edge_index

        if not adj_t.has_value():
            adj_t = adj_t.fill_value(1., dtype=dtype)
        if add_self_loops:
            adj_t = torch_sparse.fill_diag(adj_t, fill_value)

        deg = torch_sparse.sum(adj_t, dim=1)
        deg_inv_sqrt = deg.pow_(-0.5)
        deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0.)
        adj_t = torch_sparse.mul(adj_t, deg_inv_sqrt.view(-1, 1))
        adj_t = torch_sparse.mul(adj_t, deg_inv_sqrt.view(1, -1))

        return adj_t

    if is_torch_sparse_tensor(edge_index):
        assert edge_index.size(0) == edge_index.size(1)

        if edge_index.layout == torch.sparse_csc:
            raise NotImplementedError("Sparse CSC matrices are not yet "
                                      "supported in 'gcn_norm'")

        adj_t = edge_index
        if add_self_loops:
            adj_t, _ = add_self_loops_fn(adj_t, None, fill_value, num_nodes)

        edge_index, value = to_edge_index(adj_t)
        col, row = edge_index[0], edge_index[1]

        deg = scatter(value, col, 0, dim_size=num_nodes, reduce='sum')
        deg_inv_sqrt = deg.pow_(-0.5)
        deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0)
        value = deg_inv_sqrt[row] * value * deg_inv_sqrt[col]

        return set_sparse_value(adj_t, value), None

    assert flow in ['source_to_target', 'target_to_source']
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    if add_self_loops:
        edge_index, edge_weight = add_remaining_self_loops(
            edge_index, edge_weight, fill_value, num_nodes)

    if edge_weight is None:
        edge_weight = torch.ones((edge_index.size(1), ), dtype=dtype,
                                 device=edge_index.device)

    row, col = edge_index[0], edge_index[1]
    idx = col if flow == 'source_to_target' else row
    deg = scatter(edge_weight, idx, dim=0, dim_size=num_nodes, reduce='sum')
    deg_inv_sqrt = deg.pow_(-0.5)
    deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0)
    edge_weight = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    return edge_index, edge_weight


In [ ]:
class MyGCNConv(MessagePassing):

    _cached_edge_index: Optional[OptPairTensor]
    _cached_adj_t: Optional[SparseTensor]

    def __init__(
        self,
        in_channels: int,
        in_channels_edges: int,
        out_channels: int,
        improved: bool = False,
        cached: bool = False,
        add_self_loops: Optional[bool] = None,
        normalize: bool = True,
        bias: bool = True,
        **kwargs,
    ):
        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)

        if add_self_loops is None:
            add_self_loops = normalize

        if add_self_loops and not normalize:
            raise ValueError(f"'{self.__class__.__name__}' does not support "
                             f"adding self-loops to the graph when no "
                             f"on-the-fly normalization is applied")

        self.in_channels = in_channels
        self.in_channels_edges = in_channels_edges
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.add_self_loops = add_self_loops
        self.normalize = normalize

        self._cached_edge_index = None
        self._cached_adj_t = None

        self.lin = Linear(in_channels, out_channels//2, bias=False,
                          weight_initializer='glorot')
        self.lin_edges = Linear(in_channels_edges, out_channels//2, bias=False,
                          weight_initializer='glorot')

        if bias:
            self.bias = Parameter(torch.empty(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        super().reset_parameters()
        self.lin.reset_parameters()
        zeros(self.bias)
        self._cached_edge_index = None
        self._cached_adj_t = None

    def forward(self, x: torch.Tensor, edge_index: Adj,
                edge_weight: OptTensor = None) -> torch.Tensor:

        if isinstance(x, (tuple, list)):
            raise ValueError(f"'{self.__class__.__name__}' received a tuple "
                             f"of node features as input while this layer "
                             f"does not support bipartite message passing. "
                             f"Please try other layers such as 'SAGEConv' or "
                             f"'GraphConv' instead")

        if self.normalize:
            if isinstance(edge_index, torch.Tensor):
                cache = self._cached_edge_index
                if cache is None:
                    edge_index, edge_weight = gcn_norm(  # yapf: disable
                        edge_index, edge_weight, x.size(self.node_dim),
                        self.improved, self.add_self_loops, self.flow, x.dtype)
                    if self.cached:
                        self._cached_edge_index = (edge_index, edge_weight)
                else:
                    edge_index, edge_weight = cache[0], cache[1]

            elif isinstance(edge_index, SparseTensor):
                cache = self._cached_adj_t
                if cache is None:
                    edge_index = gcn_norm(  # yapf: disable
                        edge_index, edge_weight, x.size(self.node_dim),
                        self.improved, self.add_self_loops, self.flow, x.dtype)
                    if self.cached:
                        self._cached_adj_t = edge_index
                else:
                    edge_index = cache

        x = self.lin(x)
        edge_weight = self.lin_edges(edge_weight)

        # propagate_type: (x: Tensor, edge_weight: OptTensor)
        out = self.propagate(edge_index, x=x, edge_weight=edge_weight,
                             size=None)

        if self.bias is not None:
            out = out + self.bias

        return out

    def message(self, x_j: torch.Tensor, edge_weight: OptTensor) -> torch.Tensor:

        #breakpoint()
        return torch.cat([x_j, edge_weight], dim=1)

    def message_and_aggregate(self, adj_t: SparseTensor, x: torch.Tensor) -> torch.Tensor:
        return spmm(adj_t, x, reduce=self.aggr)

In [ ]:
class FraudDetectionModule(L.LightningModule):

  def __init__(self,nodes_size,edges_size, hidden_layer_size, depth, lr=1e-3, wd=5e-4):
    super().__init__()
    self.layers = []
    self.layers.append(MyGCNConv(nodes_size,edges_size, hidden_layer_size).to(DEVICE))
    for _ in range(depth-2):
      self.layers.append(MyGCNConv(hidden_layer_size,edges_size, hidden_layer_size).to(DEVICE))
    self.classifier = nn.Linear(hidden_layer_size, 2)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.lr = lr
    self.wd = wd
    self.validation_step_outputs = []
    self.f1 = BinaryF1Score()



    self.loss = nn.CrossEntropyLoss()


  def forward(self,data):
    edge_index = data.edge_index
    edge_attr = data.edge_attr
    x = data.x
    #breakpoint()

    for cnn in self.layers:
      x = self.relu(cnn(x, edge_index, edge_attr))
    return self.sigmoid(self.classifier(x))



  def training_step(self, batch, batch_idx):
    #breakpoint()

    #print("training")

    z = self.forward(batch)
    loss = self.loss(z[batch.train_mask],batch.y[batch.train_mask])
    # Logging to TensorBoard (if installed) by default
    self.log("train_loss", loss, prog_bar=True)
    return loss


  def validation_step(self, batch, batch_idx):
    #breakpoint()

    #print("validation")

    z = self.forward(batch)
    print(z.shape)
    val_loss = self.loss(z[batch.val_mask],batch.y[batch.val_mask])
    f1 = self.f1(z[batch.val_mask],batch.y[batch.val_mask])
    #wndb.log({"val_loss": val_loss,"f1-score":f1})
    self.log_dict({"val_loss": val_loss,"f1-score":f1}, prog_bar=True)


  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.wd)
    return optimizer




In [ ]:


# wndb.init(
#     project="datamining-hw4",

#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR ,
#     "weight decay": WD ,
#     "layers": DEPTH ,
#     "hidden layers size": H_SIZE
#     })
#dataset.to_device()
loader = dataset.get_dataloader()
#breakpoint()

model = FraudDetectionModule(dataset.data[0].num_node_features,dataset.data[0].edge_attr.shape[1] ,H_SIZE,DEPTH,LR,WD)
#model.to(DEVICE)

trainer = L.Trainer(deterministic=True,max_epochs=15,accelerator=ACCELERATOR,precision="16-mixed")



INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# for name, param in model.named_parameters():
    # print(f"Parameter {name} device:", param.device)

trainer.fit(model, loader, loader)
#wndb.finish()

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type             | Params
------------------------------------------------
0 | classifier | Linear           | 18    
1 | relu       | ReLU             | 0     
2 | sigmoid    | Sigmoid          | 0     
3 | f1         | BinaryF1Score    | 0     
4 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
18        Trainable params
0         Non-trainable params
18        Total params
0.000     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | classifier | Linear           | 18    
1 | relu       | ReLU             | 0     
2 | sigmoid    | Sigmoid          | 0     
3 | f1         | BinaryF1Score    | 0     
4 | loss       | CrossEntropyLoss | 0     
---------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


torch.Size([9073900, 2])


IndexError: ignored

In [ ]:
dataset.analize_data(find_empty_records=False)
print("inconsistent fraud")
print(len(inconsistent_data.query('isFraud == 1'))/len(inconsistent_data))
print("inconsistent not fraud")
print(len(inconsistent_data.query('isFraud == 0'))/len(inconsistent_data))
print("given > gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) > abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("given < gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) < abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("distinct origin name")
print(dataset.raw_data["nameOrig"].nunique())
print("distinct names")
print(len(names_distinct))
print("isFlaggedFraud and isFraud")
print(len(dataset.raw_data.query("isFlaggedFraud & isFraud"))/len(dataset.raw_data.query("isFraud")))
plot_categories(dataset.raw_data)
